In [2]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from pathlib import Path
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
import torch
from pathlib import Path
import re
import os

# Directory containing PDFs
pdf_dir = (Path.cwd()) / 'Data'

# Ensure the directory exists
if not pdf_dir.exists():
    raise ValueError(f"Directory {pdf_dir} does not exist")

# Load all PDFs from the directory
chunked_docs = []

for pdf_path in pdf_dir.glob("*.pdf"):
    loader = PyPDFLoader(str(pdf_path))
    async for page in loader.alazy_load():
        chunked_docs.append(page)

from langchain.vectorstores import FAISS
#from langchain.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

#embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

#db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="BAAI/bge-base-en-v1.5"))
db = FAISS.from_documents(chunked_docs, HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2"))

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k": 4})

class ChatGenerator:
    def __init__(self):
        self.checkpoint = (Path.cwd()).parent / 'Llama-3.2-3B-Instruct' # Set the model checkpoint path
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint) # Load tokenizer
        self.model = AutoModelForCausalLM.from_pretrained(
            self.checkpoint,
            torch_dtype=torch.bfloat16,
            device_map="cuda"  # Automatically uses GPU if available
        )
        print(self.model.config.pad_token_id)
        self.text_generator = transformers.pipeline(
            "text-generation",
            model=self.model,
            tokenizer=self.tokenizer,
            torch_dtype=torch.bfloat16,
            device_map="cuda"  # Auto-detect and use GPU
        )

    def generate_response(self, question: str):
        prompt = (
            "<|begin_of_text|>"
            "<|start_header_id|>user<|end_header_id|>"
            f"{question}\n\nContext:\n{context}"
            "<|eot_id|>"
            "<|start_header_id|>assistant<|end_header_id|>"
        )

        # Generate sequences
        sequences = self.text_generator(
            prompt,
            do_sample=True,
            top_k=2,
            num_return_sequences=1,
            eos_token_id=self.tokenizer.eos_token_id,
            truncation=True,
            max_length=12000,
        )

        # Extract and return generated responses
        for sequence in sequences:
            match = re.search(r'<\|end_header_id\|>\\n\\n(.*?)\'\}', str(sequence), re.DOTALL)
            response = match.group(1)  # Gets everything after the last '<|end_header_id|>'
        return sequences, response

# Usage example
chat_bot = ChatGenerator()
chat_history = ""

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

None


In [3]:
while True:
    question = input("Enter your question: ")
    if question.lower() == "exit":
        break
    relevant_docs = retriever.get_relevant_documents(question)
    context = "\n".join([doc.page_content for doc in relevant_docs])
    sequences, response = chat_bot.generate_response(chat_history + question + context)
    chat_history += str(sequences)
    print(response)

C:\Users\barbu\AppData\Local\Temp\ipykernel_22016\3116917095.py:5: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  relevant_docs = retriever.get_relevant_documents(question)
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


AttributeError: 'NoneType' object has no attribute 'group'